# Store parcels and blocks data in Postgres database

for a fast spatial-join of parcels to blocks.

First, create the database from terminal if it doesn't already exist:

```
createdb -U postgres parcels_blocks
psql -U postgres -d parcels_blocks -c "CREATE EXTENSION postgis;"
```

Save postgres username, password, host, port, and database name in keys.py. Then open the parcels and blocks shapefiles, then load each into tables in a PostGIS enabled database.

In [1]:
import geopandas as gpd
import psycopg2

# postgres info is stored in keys.py
from keys import pg_user, pg_pass, pg_host, pg_port, pg_db

crs = {'init':'epsg:4326'}

## Load and project the spatial data

In [2]:
%%time
# load blocks shapefile and project to 4326
blocks = gpd.read_file('data/tl_2010_06_tabblock10/')
blocks = blocks.to_crs(crs)
print(len(blocks))

710145
Wall time: 2min 12s


In [3]:
%%time
# load parcels shapefile, drop any that lack geometry, and project to 4326
parcels = gpd.read_file('data/Parcels/')
parcels = parcels.dropna(subset=['geometry'])
parcels = parcels.to_crs(crs)
print(len(parcels))

1956207
Wall time: 5min 36s


In [4]:
# srid is the numeric spatial reference ID that PostGIS uses
assert crs == blocks.crs == parcels.crs
srid = crs['init'].strip('epsg:')
srid

'4326'

## Clean up the parcels data

In [5]:
# convert integer columns to int
to_int = ['PARCEL_ID', 'DEVELOPMEN', 'COUNTY_ID', 'ZONE_ID', 'PROPORTION', 'TAX_EXEMPT', 'ID']
for col in to_int:
    parcels[col] = parcels[col].astype(int)

In [6]:
# drop unused columns and calculate centroids
parcels = parcels.drop(columns=['CENTROID', 'X', 'Y'])
parcels['centroid'] = parcels.centroid

## Insert blocks and parcels into PostGIS database

In [7]:
# connect to the postgres database
connection = psycopg2.connect(database=pg_db,
                              user=pg_user,
                              password=pg_pass,
                              host=pg_host,
                              port=pg_port)
cursor = connection.cursor()

In [8]:
# list all tables
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)'")
cursor.fetchall()

[('spatial_ref_sys',), ('blocks',), ('parcels',)]

### add blocks table and data

In [9]:
# drop blocks table if it already exists, then create blocks table
cursor.execute("DROP TABLE IF EXISTS blocks")
cursor.execute("""CREATE TABLE blocks (id SERIAL PRIMARY KEY,
                                       geoid VARCHAR NOT NULL)""")
cursor.execute("SELECT AddGeometryColumn ('blocks', 'geom', %s, 'MULTIPOLYGON', 2)", [srid])
cursor.execute("CREATE INDEX block_index ON blocks USING GIST(geom)")
connection.commit()

In [10]:
%%time
cursor.execute("DELETE FROM blocks")

# insert each block into the blocks table one at a time
for label, row in blocks.iterrows():
    geoid = row['GEOID10']
    geometry_wkt = row['geometry'].wkt
    
    query = """INSERT INTO blocks (geoid, geom) 
               VALUES (%s, ST_Multi(ST_GeomFromText(%s, %s)))"""
    data = (geoid, geometry_wkt, srid)
    cursor.execute(query, data)

connection.commit()

Wall time: 6min 49s


### add parcels table and data

In [11]:
# drop parcels table if it already exists, then create parcels table
cursor.execute("DROP TABLE IF EXISTS parcels")
cursor.execute("""CREATE TABLE parcels (id SERIAL PRIMARY KEY,
                                        parcel_id INTEGER NOT NULL)""")
cursor.execute("SELECT AddGeometryColumn ('parcels', 'geom', %s, 'POINT', 2)", [srid])
cursor.execute("CREATE INDEX parcel_index ON parcels USING GIST(geom)")
connection.commit()

In [12]:
%%time
cursor.execute("DELETE FROM parcels")

# insert each parcel into the parcels table one at a time
for label, row in parcels.iterrows():
    parcel_id = row['PARCEL_ID']
    geometry_wkt = row['centroid'].wkt
    
    query = """
            INSERT INTO parcels (parcel_id, geom)
            VALUES (%s, ST_GeomFromText(%s, %s))
            """
    data = (parcel_id, geometry_wkt, srid)
    cursor.execute(query, data)

connection.commit()

Wall time: 10min 9s


### optimize the database

In [13]:
%%time
# vacuum and analyze the database to optimize it after building indices and inserting rows
original_isolation_level = connection.isolation_level
connection.set_isolation_level(0)
cursor.execute("VACUUM ANALYZE")
connection.commit()
connection.set_isolation_level(original_isolation_level)

Wall time: 1.33 s


## Verify the data insertion

In [14]:
# verify the SRIDs
cursor.execute("""SELECT
                  Find_SRID('public', 'blocks', 'geom') as blocks_srid,
                  Find_SRID('public', 'parcels', 'geom') as parcels_srid""")
cursor.fetchall()

[(4326, 4326)]

### verify the blocks

In [15]:
cursor.execute("SELECT count(*) AS exact_count FROM blocks")
rows = cursor.fetchall()
rows[0][0]

710145

In [16]:
cursor.execute("SELECT geoid, ST_AsText(geom) FROM blocks LIMIT 3")
rows = cursor.fetchall()
gpd.GeoDataFrame(rows, columns=['GEOID', 'geometry'])

,GEOID,geometry
0,060014100002024,"MULTIPOLYGON(((-122.124886 37.751635,-122.1247..."
1,060014073002017,"MULTIPOLYGON(((-122.210245 37.770097,-122.2101..."
2,060014081002011,"MULTIPOLYGON(((-122.18212 37.794933,-122.18183..."


### verify the parcels

In [17]:
cursor.execute("SELECT count(*) AS exact_count FROM parcels")
rows = cursor.fetchall()
rows[0][0]

1956207

In [18]:
cursor.execute("""SELECT parcel_id, ST_AsText(geom)
                  FROM parcels LIMIT 3""")
rows = cursor.fetchall()
gpd.GeoDataFrame(rows, columns=['parcel_id', 'geometry'])

,parcel_id,geometry
0,229116,POINT(-121.79562028956 37.6553791226699)
1,244166,POINT(-121.713003932049 37.7172768065733)
2,202378,POINT(-122.014198767807 37.6552595604717)


## All done

In [19]:
cursor.close()
connection.close()